# "Answering a Twitter Question"
> "Basic monte carlo simulations for Taleb's kappa"

# Answering a Twitter question
Someone on Twitter [tried to Monte Carlo simulate](https://twitter.com/DRyabich/status/1271188956785319941/photo/1) Taleb's $\kappa$ (from [Statistical consequences of fat tails](https://arxiv.org/abs/2001.10488)), for a normal distribution, but ran into issues. Maybe the following exposition with standard normal variables (Gaussian with mean 0, variance 1) will help.

Taleb's $\kappa$ is defined for a sum of $n$ random variables, $S_n = \sum_{i=1}^n X_n$:

$$\kappa_{n} = \kappa(n, n+1) = 2 - \frac{\log(n+1) - \log(n)}{\log \left ( \frac{M(n+1)}{M(n)} \right )}$$

where

$$M(n) = \mathbb{E}(\left | S_n - \mathbb{E}(S_n) \right |)$$

First, let's implement $\mathbb{M}(n)$. To unwrap the formula, $S_n$ is the sum of $n$ standard normal variables. As we know, [the sum of normal variables is normal](https://en.wikipedia.org/wiki/Sum_of_normally_distributed_random_variables) and its variance is equal to the sum of the variances, and mean equal to the sum of the means. In other words, $S_n \sim N(0, n)$.

But then $\mathbb{E}(S_n) = 0$ and  the standard deviation (the `scale` parameter in `scipy.stats.norm`) is $\sqrt{n}$.

To calculate $\mathbb{M}(n)$ via monte carlo, we can thus take a large sample of this $S_n$, apply the absolute value function to it, and calculate the sample mean.

In [4]:
import numpy as np
from scipy.stats import norm

n_precision = 1_000_000

def M(n):
    # Since we take the variables to be N(0, 1), S_i ~ N(0, sqrt(i)).
    # So E[S_i] = 0, which means that E[|S_i - E[S_i]|] = E[|S_i|].
    # So all we need to do is take n_precision samples of S_i,
    # apply the absolute value function, and then take the mean
    return np.mean(np.abs(norm.rvs(scale=np.sqrt(n), size=n_precision)))

[M(1000) for _ in range(10)]

[25.206322069310957,
 25.23577354150956,
 25.228601059383166,
 25.202142323958526,
 25.254476571927498,
 25.23965574816039,
 25.255859740411974,
 25.2345045620554,
 25.23934429964762,
 25.22352113697182]

And dividing `M(n+1)` by `M(n)`:

In [6]:
[M(1001) / M(1000) for _ in range(10)]

[1.0021439622692407,
 1.0003524893992,
 1.0002437888464653,
 0.9999707169726871,
 1.000976732064089,
 1.0001976924363172,
 1.0000014696860589,
 0.9983938903976746,
 0.9999057817654673,
 0.9988688635204259]

Which tends to 1 quite neatly, as expected. Note that it's not possible to plug this into the formula for $\kappa$, since $\log(1) = 0$, and we can't divide by 0!